In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt
import csv
import os

Import dataset

In [10]:
#==== Import Dataset ====
dataset = pd.read_csv('/content/drive/MyDrive/PercobaanTA/Dataset_KUKA_baru_bersih.csv')

pose = dataset.iloc[:, 6:]   # Input labels (last six columns)
jointsAngle = dataset.iloc[:, :6]   # Output features (first six columns)

pose_columns = ['Position x', 'Position y', 'Position z', 'Roll', 'Pitch', 'Yaw']  # Replace with your actual column names
jointsAngle_columns = ['Joint 1', 'Joint 2', 'Joint 3', 'Joint 4', 'Joint 5', 'Joint 6']  # Replace with your actual column names

#==== Split Dataset ====
split_index = int(0.8 * len(dataset))


jointsAngle_train = jointsAngle[:split_index].values
jointsAngle_test = jointsAngle[split_index:].values
pose_train = pose[:split_index].values
pose_test = pose[split_index:].values


Eksperiment configuration

In [11]:
neuron = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 150, 200, 250,300]
windowSize = [1, 4]

for windowSizei in windowSize:
  #==== Generate time series dataset ====
  windowed_pose_train = []
  windowed_jointsAngle_train = []

  windowed_pose_test = []
  windowed_jointsAngle_test = []

  for i in range(len(pose_train) - windowSizei + 1):
      windowed_pose_train.append(pose_train[i : i + windowSizei])
      windowed_jointsAngle_train.append(jointsAngle_train[i + windowSizei - 1])

  for i in range(len(pose_test) - windowSizei + 1):
      windowed_pose_test.append(pose_test[i : i + windowSizei])
      windowed_jointsAngle_test.append(jointsAngle_test[i + windowSizei - 1])

  windowed_pose_train = np.array(windowed_pose_train)
  windowed_jointsAngle_train = np.array(windowed_jointsAngle_train)

  windowed_pose_test = np.array(windowed_pose_test)
  windowed_jointsAngle_test = np.array(windowed_jointsAngle_test)

  for neuroni in neuron:
    #Folder for saving the data
    folder_path = '/content/drive/MyDrive/PercobaanTA/RNN/RNN'+'_'+str(windowSizei)+'_3_'+str(neuroni)
    if os.path.exists(folder_path):
          continue
    os.makedirs(folder_path, exist_ok=True)



    #==== LSTM Model ====
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.SimpleRNN(neuroni, activation='relu',  input_shape=(windowSizei, 6), return_sequences=True))
    model.add(tf.keras.layers.SimpleRNN(neuroni, activation='relu', return_sequences=True))
    model.add(tf.keras.layers.SimpleRNN(neuroni, activation='relu'))
    #model.add(Flatten())
    model.add(Dense(6))

    model.compile(optimizer='adam', loss='mse')

    #==== Target Function ====
    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_loss') <= 50): # Experiment with changing this value
          print("\nReached loss less than 0.2 so cancelling training!")
          self.model.stop_training = True

    #==== Train Model ====
    callbacks = myCallback()


    #history = model.fit(pose_train, jointsAngle_train, epochs=3000, batch_size=32, callbacks=[callbacks])

    history = model.fit(windowed_pose_train, windowed_jointsAngle_train, epochs=3000, batch_size=32, validation_data=(windowed_pose_test, windowed_jointsAngle_test), callbacks=[callbacks])

    # Plot loss data
    loss_data = history.history['loss']
    loss_data_plotted = loss_data[1:]

    val_loss_data = history.history['val_loss']
    val_loss_data_plotted = val_loss_data[1:]

    plt.figure()
    plt.plot(range(len(loss_data_plotted)), loss_data_plotted, label='Loss')
    plt.plot(range(len(val_loss_data_plotted)), val_loss_data_plotted, label='Evaluation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss value')
    plt.title('Loss')
    plt.legend()
    # Save loss plot
    plt.savefig(os.path.join(folder_path, 'loss.png'))
    plt.show()

    print(val_loss_data[-1])
    print(loss_data[-1])

    # Write loss and eval loss value
    loss_file = folder_path + '/loss.txt'

    # Open the file in write mode
    with open(loss_file, 'w') as file:
        # Write content to the file
        file.write('Loss = '+str(loss_data[-1])+'\n')
        file.write('Eval loss = '+str(val_loss_data[-1])+'\n')
        file.write('Epoch = '+str(len(loss_data))+'\n')

    #==== Prediction Test ====
    pose_predict = model.predict(windowed_pose_test)

    print(pose_predict)

    # Save prediction data into csv file
    header = ['Joint 1', 'Joint 2', 'Joint 3', 'Joint 4', 'Joint 5', 'Joint 6']
    rows = pose_predict.tolist()  # Convert predictions to a list
    csv_file = folder_path + '/Predictions_KUKA.csv'

    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(rows)

    i = 0
    jointsAngle_test = jointsAngle[split_index:]
    joint = 1
    # Plot joint angle test data and joint angle prediction
    for column in jointsAngle_test:
        input_data = jointsAngle_test[column]
        predict_data = pose_predict[:, i]
        i = i+1
        plt.figure()
        plt.plot(range(len(input_data)), input_data, label=column)
        plt.plot(range(len(predict_data)), predict_data, label = column + ' prediction')
        plt.xlabel('Index')
        plt.ylabel('Joint Angle (degree)')
        plt.title(column)
        plt.legend()
        # Save plot to folder
        plt.savefig(os.path.join(folder_path, 'A'+str(joint)+'.png'))
        joint = joint+1
        plt.show()



In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
